In [1]:
import pandas as pd

In [2]:
def first_initials_to_final(
    string, split_char=",", inner_split=" ", join_char=", ", inner_join=" "
):
    out = []

    for s in string.split(split_char):
        meta = s.split(inner_split)

        last_name = meta[-1].strip()
        first_name = "".join(meta[:-1])

        out.append(f"{last_name}{inner_join}{first_name}")

    return join_char.join(out)


def semicolon_to_final(
    string, split_char=";", inner_split=",", join_char=", ", inner_join=" "
):
    out = []

    for s in string.split(split_char):
        meta = s.split(inner_split)

        last_name = meta[0].strip()

        first_name = ("".join(meta[1:])).replace(" ", "")

        out.append(f"{last_name}{inner_join}{first_name}")

    return join_char.join(out)


def apply_by_row(rows, semicolon=None):
    if semicolon is None:
        semicolon = [None] * len(rows)

    out = []
    for row, _semi in zip(rows, semicolon):
        semi = ";" in row if _semi is None else _semi

        if semi:
            out.append(semicolon_to_final(row))
        else:
            out.append(first_initials_to_final(row))

    return out

In [3]:
FY = 2025
Quarter = 1


def query_fy_quarter(df):
    return df[(df["Fiscal Year"] == FY) & (df["Quarter"] == Quarter)]


path = "./Publications_and_presentations_group04.xlsx"

df_papers = (
    pd.read_excel(path, "papers").dropna(how="all").pipe(query_fy_quarter)
    #    .dtypes({'Fiscal Year': 'datetime'})
)

df_talks = (
    pd.read_excel(path, "talks")
    .dropna(how="all")
    .pipe(query_fy_quarter)
    .astype({"Date": "datetime64[ns]"})
)

# papers

In [4]:
def paper_format(df):
    data = []
    for _, row in df.iterrows():
        out = "{authors}, {title}, {ref}".format(
            authors=row["Authors"], title=row["Title"], ref=row["Reference"]
        )

        if not row.isna()["Doi"]:
            out += ", {}".format(row["Doi"])
        data.append(out)
    return data

In [5]:
df_papers

,Fiscal Year,Quarter,Title,Authors,Reference,Doi,Notes
41,2025.0,1.0,Progress in Development of Characterization Ca...,"M. Carter, H. G. T. Nguyen, A. J. Allen, F. Yi...","Journal of CO2 Utilization 2024, 89, 102975",https://doi.org/10.1016/j.jcou.2024.102975,NaN
42,2025.0,1.0,Per- and Polyfluoroalkyl Substances – Non-Targ...,"Place BJ, Reiner JL, Ragland JM, Rodowa AE, Bu...",NIST Internal Report,https://doi.org/10.6028/NIST.IR.8544,NaN
43,2025.0,1.0,The Avogadro constant is not the defining cons...,"R. N. Kacker, K. K. Irikura","Metrologia, 62, 013003 (2025)",https://doi.org/10.1088/1681-7575/ada7fc,NaN


In [6]:
# Fix semicolon format for any rows
t = df_papers.assign(Authors=lambda x: apply_by_row(x["Authors"], None))

In [7]:
for x in paper_format(t):
    print("*", x)

* Carter M., Nguyen H.G.T., Allen A.J., Yi F., Yang W-CD., Baumann A.E., McGivern W.S., Manion J.A., Kuzmenko I., Sinas Z., Wentz C.M., Wenny M., Siderius D.W., Zee R.D.van, Stafford C.M., Brown C.M., Progress in Development of Characterization Capabilities to Evaluate Candidate Materials for Direct Air Capture Applications, Journal of CO2 Utilization 2024, 89, 102975, https://doi.org/10.1016/j.jcou.2024.102975
* BJ Place, JL Reiner, JM Ragland, AE Rodowa, CQ Burdette, AA Cuthbertson, CA Rimmer, JR Kucklick, N Mahynski, Per- and Polyfluoroalkyl Substances – Non-Targeted Analysis Interlaboratory Study Final Report, NIST Internal Report, https://doi.org/10.6028/NIST.IR.8544
* Kacker R.N., Irikura K.K., The Avogadro constant is not the defining constant of the mole, Metrologia, 62, 013003 (2025), https://doi.org/10.1088/1681-7575/ada7fc


# Talks

In [8]:
def talk_format(df):
    df = df.assign(
        month_year=lambda x: x["Date"].dt.month_name()
        + " "
        + x["Date"].dt.year.astype(str)
    )

    data = []
    for _, row in df.iterrows():
        out = "{authors}, {title}, {ref}, {loc}, {date} ({Type})".format(
            authors=row["Authors"],
            title=row["Title"],
            ref=row["Conference"],
            loc=row["Location"],
            date=row["month_year"],
            Type=row["Type"],
        )

        data.append(out)
    return data

In [9]:
for x in talk_format(
    df_talks.assign(Authors=lambda x: apply_by_row(x["Authors"], None))
):
    print("*", x)

* Chalk S., Evans J.D., Iacomi P., Kaskel S., Ohrstrom L., Siderius D.W., Silvestre J., Speybroeck V.Van, Tanaka H., IUPAC Adsorption Information Format: A universal standard archive file for adsorption data, AIChE Annual Meeting, San Diego, CA, October 2024 (Talk)
* Siderius D.W., Hatch H.W., Shen andV.K., Flat-histogram Monte Carlo Simulation of Binary Adsorption in Crystalline Porous Adsorbents, AIChE Annual Meeting, San Diego, CA, October 2024 (Talk)


In [10]:
df_talks.Date.dt.month_name() + " " + df_talks.Date.dt.year.astype(str)

58    October 2024
59    October 2024
Name: Date, dtype: object